In [1]:
# add default values for parameters here

In [2]:
# Parameters
product = "./4. Advanced features/4.9. Virtual Devices.ipynb"


# Devices and Virtual Devices

In [3]:
import numpy as np
from dataclasses import replace
from pulser.devices import Device, VirtualDevice, Chadoq2, MockDevice
from pulser.channels import Rydberg, Raman, Microwave
from pulser import Pulse, Sequence, Register

## Physical devices

To perform any computation using Pulser, it is necessary to choose a device. For convenience, some examples of typical physical devices are included and can be accessed via `pulser.devices`.  
These devices are instances of the `Device` class. They are constrained by physical considerations and all their parameters are defined.

As an example, we present below the specifications of the physical device `Chadoq2`, which can be accessed via the `Device.print_specs()` method.

In [4]:
# Display Chadoq2's specifications
Chadoq2.print_specs()

----------------------
Chadoq2 Specifications
----------------------

Register parameters:
 - Dimensions: 2D
 - Rydberg level: 70
 - Maximum number of atoms: 100
 - Maximum distance from origin: 50 μm
 - Minimum distance between neighbouring atoms: 4 μm
 - Maximum layout filling fraction: 0.5
 - SLM Mask: Yes

Channels:
 - 'rydberg_global': Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'rydberg_local': Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'raman_local': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Mini

## Virtual Devices

### Converting a Device into a Virtual Device

However, we sometimes want to perform the computations on a more permissive device, a device that would have more dimensions or more atoms for instance, or more types of channels. This can be done on an emulator behaving like a device. The `VirtualDevice` class is useful to define such an emulator, a virtual device.

Let's start by configuring a virtual device having the same parameters as `Chadoq2`. To do this, we use the `Device.to_virtual()` method that creates a virtual device from a physical one. 

In [5]:
# Converting the Device object in a VirtualDevice object
VirtualChadoq = Chadoq2.to_virtual()
print(VirtualChadoq)

VirtualDevice(name='Chadoq2', dimensions=2, rydberg_level=70, min_atom_distance=4, max_atom_num=100, max_radial_distance=50, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=False, channel_ids=('rydberg_global', 'rydberg_local', 'raman_local'), channel_objects=(Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fix

### Changing parameters of a virtual device with `dataclasses.replace()`

As proposed earlier, a virtual device gives us the possibility to modify the parameters of a physical device. The function `dataclasses.replace()` can be used to create a new `VirtualDevice` having some parameters changed.  
For example, simulations can be run on a virtual device having the same properties as `Chadoq2` but allowing working in 3 dimensions.

In [6]:
# Adding a dimension to the emulator
VirtualChadoq3D = replace(VirtualChadoq, dimensions=3)
print(VirtualChadoq3D)

VirtualDevice(name='Chadoq2', dimensions=3, rydberg_level=70, min_atom_distance=4, max_atom_num=100, max_radial_distance=50, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=False, channel_ids=('rydberg_global', 'rydberg_local', 'raman_local'), channel_objects=(Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fix

### Unrealistic parameters of a virtual device

Among the parameters defined in a `VirtualDevice`, some are called unrealistic as they do not refer to something physically implementable.  
For instance, it is possible to change the Rydberg level used in the simulation or to call a channel multiple times in the same sequence. Modifying the Rydberg level has an impact on the coefficient $C_6$ (see [here](https://github.com/pasqal-io/Pulser/blob/develop/pulser-core/pulser/devices/interaction_coefficients/C6_coeffs.json) for the correspondance between the Rydberg level and $C_6$ coefficient).

In [7]:
# Changing the Rydberg level
VirtualChadoq3D.change_rydberg_level(61)
print(VirtualChadoq3D)

VirtualDevice(name='Chadoq2', dimensions=3, rydberg_level=61, min_atom_distance=4, max_atom_num=100, max_radial_distance=50, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=False, channel_ids=('rydberg_global', 'rydberg_local', 'raman_local'), channel_objects=(Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fix

In [8]:
# Enable the multiple declaration of a channel in a sequence
VirtualChadoq3D = replace(VirtualChadoq3D, reusable_channels=True)
# Creating a square register
reg = Register.square(4, spacing=5)  # 4x4 array with atoms 5 um apart
# Building a sequence with the register and the virtual device
seq = Sequence(reg, VirtualChadoq3D)
# Declare twice the channel "ram_loc"
seq.declare_channel("ch0", "raman_local")
seq.declare_channel("ch1", "raman_local")
# Show the declared channels
print(seq.declared_channels)

{'ch0': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'digital'), 'ch1': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'digital')}


### Building your own virtual device

More generally, the `VirtualDevice` class is more permissive than the `Device` class. In fact, some parameters can be left undefined and simpler devices can be built with a `VirtualDevice` instance. A virtual device only needs a `name`, a `dimension` and a `rydberg_level` to be initialized.

In [9]:
BasicVirtualDevice = VirtualDevice(
    name="BasicMockDevice",
    dimensions=2,
    rydberg_level=61,
)
print(BasicVirtualDevice)

VirtualDevice(name='BasicMockDevice', dimensions=2, rydberg_level=61, min_atom_distance=0, max_atom_num=None, max_radial_distance=None, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=True, channel_ids=(), channel_objects=(), dmm_objects=(DMM.Global(Max Absolute Detuning: None, Max Amplitude: 0, Clock period: 1 ns, Minimum pulse duration: 1 ns, Maximum pulse duration: 100000000 ns, Basis: 'ground-rydberg'),))


### Defining the channels of your device

Nevertheless, to perform computations, channels have to be defined. The channels enabled on the device are defined in `channel_objects`. Their IDs can be defined in `channel_ids`, but if no IDs are provided, they will be automatically generated as `{channeltype}_adressing`.  
For example, the `Rydberg.Global` channel is automatically named `rydberg_global` in the specifications of `Chadoq2`.

In [10]:
# This basic device can be used for digital quantum computing
DigitalQCVirtualDevice = replace(
    BasicVirtualDevice,
    channel_ids=(
        "ryd_loc",
        "ram_loc",
    ),
    channel_objects=(
        Rydberg.Local(None, None, max_duration=None),
        Raman.Local(None, None, max_duration=None),
    ),
)
print(DigitalQCVirtualDevice)

VirtualDevice(name='BasicMockDevice', dimensions=2, rydberg_level=61, min_atom_distance=0, max_atom_num=None, max_radial_distance=None, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=True, channel_ids=('ryd_loc', 'ram_loc'), channel_objects=(Rydberg.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'digital')), dmm_objects=(DMM.Global(Max Absolute Detuning: None, Max Amplitude: 0, Clock period: 1 ns, Minimum pulse duration: 1 ns, Maximum pulse duration: 100000000 ns, Basis: 'ground-rydberg'),))


### A built-in example of a virtual device: the `MockDevice`

Actually, there exists a virtual device having all the channels already implemented, with no constraints on the number of atoms, the distance between them. This virtual device is the `MockDevice`.

In [11]:
MyMockDevice = replace(MockDevice, name="MyMockDevice")
print(MyMockDevice)

VirtualDevice(name='MyMockDevice', dimensions=3, rydberg_level=70, min_atom_distance=0.0, max_atom_num=None, max_radial_distance=None, interaction_coeff_xy=3700.0, supports_slm_mask=True, max_layout_filling=0.5, max_sequence_duration=None, max_runs=None, reusable_channels=True, channel_ids=('rydberg_global', 'rydberg_local', 'raman_global', 'raman_local', 'mw_global'), channel_objects=(Rydberg.Global(Max Absolute Detuning: None, Max Amplitude: None, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Raman.Global(Max Absolute Detuning: None, Max Amplitude: None, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'digital'), Raman.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Mini